# Cleverly Challenge

This works as my submission of the cleverly challenge.
The challenge consists in the [Amazon products review dataset](http://jmcauley.ucsd.edu/data/amazon/) (Clothing, Shoes and Jewlry)

This dataset consists 278677 reviews about this types o items where each row in the dataset has the followng information:
- reviewrID - ID of the reviewer (39387 unique values)
- asin - ID of the product (23033 unique values)
- reviewerName - name of the reviewer
- helpful - a tuple containing the number of "Yes" votes, and the total number of votes
- reviewText - The main body text of the review
- overall - The rating the user gave the product
- summary - A short summary of the review
- UnixReviewTime - Unix timestamp of the review
- reviewTime - Time of the review (raw)

## Importante: 
- A Helpfullness é ("Yes" votes, total number of votes) então se calhar aqui tenho que fazer tipo a proporção de helpfulness, em que se calhar uma helpfull question é uma que tenha pelo menos 50% de votos sendo "Yes"
- Hugging faces transformers requires TF 2.0 or pytorch

Posso tentar ver net promoter score dos produtos e fazer tipo um histogram disso (https://www.kaggle.com/shekhart47/research-and-predict-amazon-kindle este dataset tem ai isso)

Posso tentar fazer sentiment analysis?

Ver quais é que são as palavras mais frequentes nem perguntas uteis (e em perguntas que tenham tipo rating de 5) (lembrar de tirar stopwords, fazer lemmantization, stemming, etc)
    Posso fazer um wordcloud disto por exemplo
    Ver também com o TF-IDF para conseguir perceber a "novelty" dessas palavras
    
Questions:

Is there a correlation between rating of the product and helpfulness of review?
    Aqui posso tentar 3 coisas:
        - Ver a correlacao do overall com o number of "Yes" votes
        - Ver a correlacao do overall com o total number of votes
        - ver a correlacao do overall com a proporcao de "Yes" votes
        - Tentar ver isto tambem com anova ? https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection
        - Tentar ver estas coisas todas mas descartando as reviews que não tem nenhum vote sequer
        
        
Some more questions:
- what are the most common words in helpfull/good reviews/products ?
- Do good products have better reviews? (this should confirm the answer of the correlation
- Are helpfull reviews longer or shorter?
- Net promoter score stuff

Criar uma correlation matrix entre helpfulness/overall e outras features tipo numero de palavras

In [115]:
# Install HuggingFace's transformer packages to use DistilBert
!pip install transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp37-none-any.whl size=884633 sha256=c94fc80871328531f123627eed25b2ddb62c97d93193ec2cf316f5a1a1deb760
  Stored in directory: C:\Users\lsfischer\AppData\Local\pip\Cache\wheels\6d\ec\1a\21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [68]:
# importing required libraries
import ast
import itertools
import transformers
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# loading the dataset
df = pd.read_csv("../data/Clothing_Shoes_and_Jewelry_5.csv")

# quick check of the data
print(df.shape)
df.head()

(278677, 9)


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0000031887,"[0, 0]",5,This is a great tutu and at a really great pri...,"02 12, 2011",A1KLRMWW2FWPL4,"Amazon Customer ""cameramom""",Great tutu- not cheaply made,1297468800
1,0000031887,"[0, 0]",5,I bought this for my 4 yr old daughter for dan...,"01 19, 2013",A2G5TCU2WDFZ65,Amazon Customer,Very Cute!!,1358553600
2,0000031887,"[0, 0]",5,What can I say... my daughters have it in oran...,"01 4, 2013",A1RLQXYNCMWRWN,Carola,I have buy more than one,1357257600
3,0000031887,"[0, 0]",5,"We bought several tutus at once, and they are ...","04 27, 2014",A8U3FAMSJVHS5,Caromcg,"Adorable, Sturdy",1398556800
4,0000031887,"[0, 0]",5,Thank you Halo Heaven great product for Little...,"03 15, 2014",A3GEOILWLK86XM,CJ,Grammy's Angels Love it,1394841600


It is always good to first check on missing values so as to know if this might be a problem or not

In [69]:
df.isna().sum()

asin                0
helpful             0
overall             0
reviewText         24
reviewTime          0
reviewerID          0
reviewerName      469
summary             1
unixReviewTime      0
dtype: int64

There are few missing values in the dataset, with most of them beeing in the reviewers name, which we might even discard this feature so no harm done there.

The `reviewText` column has 24 missing values (a small portion) which could just be discarded. However, looking at these rows we see that even though they don't have `reviewText` they still have the `summary` column, which we can still use to get some insight.

In [70]:
# Checking the rows in which the reviewText is not present
df[df["reviewText"].isna()]

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
1806,B0000ATP8L,"[0, 0]",5,NaN,"06 28, 2014",A21OBYW0N5ENS7,"Bassocantor "" OracleMagician""",&#8730; 20 YEARS and Still Going Strong!,1403913600
9827,B00074KYC8,"[70, 70]",4,NaN,"02 2, 2012",A2P4DZ9LG66F7M,jcazz,"Very nice watch, looks great, wears well!",1328140800
12528,B0007XA0FU,"[4, 4]",2,NaN,"09 28, 2013",A6SNEHZIBFK4V,Living Life In The Midwest,Ruined a classic by changing the design.,1380326400
15292,B0009HAIGK,"[1, 1]",5,NaN,"12 27, 2013",A2KW2LCO5QUWOI,"Jerrod Kowalski ""Jerrod Kowalski""",2XL for size 13 cross-trainer shoes,1388102400
17610,B000AOMDFG,"[2, 43]",5,NaN,"07 10, 2010",A2T0QHP0GKI4KG,"Estina ""Estina Anghel""",Love This Product,1278720000
24830,B000FL1M1K,"[0, 0]",5,NaN,"04 5, 2013",A1H3D6C2VBGOOZ,Shopper playgirl,Jackie O glasses at a low cost,1365120000
25896,B000GAYQJ0,"[34, 35]",5,NaN,"11 7, 2012",A2I6GCGJD0Q6PY,H. Stevens,how to adust the buckle on the Casio A158W-1,1352246400
28236,B000H92UKM,"[43, 47]",5,NaN,"05 12, 2013",AERTVDQPAD7WC,"William Doerner ""Will""",Who makes a video for shoe trees??,1368316800
31232,B000K2J4R8,"[0, 0]",5,NaN,"06 12, 2014",AT1YZX8OLUEWX,pampaints,"soft, cozy and warm - but it's teal.",1402531200
35324,B000N40W9W,"[0, 0]",5,NaN,"12 26, 2013",A2RM5ZSZJQL7WY,Susan E Crangle,Fantastic and educational,1388016000


Looking at this segment of the data we notice that the `reviewerName` of index 1806 has a non-ascii character. It might be best to remove this characters all together

In [66]:
# Specify the non-ascii characters. Non ascii chars go from 0-31 and from 128-256
ascii_ranges = itertools.chain(range(32), range(128, 257))

# Create a lookup table specifying that non-ascii chars should be translated to empty strings
tbl = dict.fromkeys(ascii_ranges, u"")

# Actually convert the chars
# str.translate uses C-level table lookup so it's much faster than iterating over every char and replacing them
df["reviewerName"] = df["reviewerName"].str.translate(tbl)
df["reviewText"] = df["reviewText"].str.translate(tbl)
df["summary"] = df["summary"].str.translate(tbl)

Having done a quick check on the data, we can move to start answeing some questions

First let's split up the helpfulness column into 3 different columns:
    - `num_yes_votes`: Number of "Yes" votes
    - `num_votes`: Total number of votes
    - `prop_yes_votes`: Proportion of "Yes" votes in the reiview
    
The `helpful` column is stored as the string representation of a list (from beeing stored in a `.csv` file). We first need to convert back to a list and then split the values of the list

In [67]:
# Evaluating the string representation of the list to convert it to an actual list
df["helpful"] = df["helpful"].apply(lambda str_list: ast.literal_eval(str_list))

# Split the helpful column into num_yes_votes and num_votes
# Splitting this way is faster the applying a fuction to every row to get the first and second element
df[['num_yes_votes','num_votes']] = pd.DataFrame(df["helpful"].values.tolist(), index= df.index)

# Creating the prop_yes_votes columns. Where there is no votes just leave at zero (avoid dividing by zero)
df["prop_yes_votes"] = np.where(df["num_votes"] == 0, 0, df["num_yes_votes"] / df["num_votes"])

Adding new feats

In [28]:
#Adding new features
# count the number of words in the text, including stopwords
df["num_words"] = df["reviewText"].str.count("\w+") # Count the number of words in the body


def clean_sentence(s):
    """Given a sentence remove its punctuation and stop words"""
    
    stop_words = set(stopwords.words('english'))
    s = s.translate(str.maketrans('','',string.punctuation)) # remove punctuation
    tokens = word_tokenize(s)
    cleaned_s = [w for w in tokens if w not in stop_words] # removing stop-words
    return " ".join(cleaned_s)

In [29]:
df[df["reviewText"].isna()]
#df.iloc[1806, :]

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,num_yes_votes,num_votes,prop_yes_votes,num_words
1806,B0000ATP8L,"[0, 0]",5,NaN,"06 28, 2014",A21OBYW0N5ENS7,"Bassocantor "" OracleMagician""",&#8730; 20 YEARS and Still Going Strong!,1403913600,0,0,0.000000,NaN
9827,B00074KYC8,"[70, 70]",4,NaN,"02 2, 2012",A2P4DZ9LG66F7M,jcazz,"Very nice watch, looks great, wears well!",1328140800,70,70,1.000000,NaN
12528,B0007XA0FU,"[4, 4]",2,NaN,"09 28, 2013",A6SNEHZIBFK4V,Living Life In The Midwest,Ruined a classic by changing the design.,1380326400,4,4,1.000000,NaN
15292,B0009HAIGK,"[1, 1]",5,NaN,"12 27, 2013",A2KW2LCO5QUWOI,"Jerrod Kowalski ""Jerrod Kowalski""",2XL for size 13 cross-trainer shoes,1388102400,1,1,1.000000,NaN
17610,B000AOMDFG,"[2, 43]",5,NaN,"07 10, 2010",A2T0QHP0GKI4KG,"Estina ""Estina Anghel""",Love This Product,1278720000,2,43,0.046512,NaN
24830,B000FL1M1K,"[0, 0]",5,NaN,"04 5, 2013",A1H3D6C2VBGOOZ,Shopper playgirl,Jackie O glasses at a low cost,1365120000,0,0,0.000000,NaN
25896,B000GAYQJ0,"[34, 35]",5,NaN,"11 7, 2012",A2I6GCGJD0Q6PY,H. Stevens,how to adust the buckle on the Casio A158W-1,1352246400,34,35,0.971429,NaN
28236,B000H92UKM,"[43, 47]",5,NaN,"05 12, 2013",AERTVDQPAD7WC,"William Doerner ""Will""",Who makes a video for shoe trees??,1368316800,43,47,0.914894,NaN
31232,B000K2J4R8,"[0, 0]",5,NaN,"06 12, 2014",AT1YZX8OLUEWX,pampaints,"soft, cozy and warm - but it's teal.",1402531200,0,0,0.000000,NaN
35324,B000N40W9W,"[0, 0]",5,NaN,"12 26, 2013",A2RM5ZSZJQL7WY,Susan E Crangle,Fantastic and educational,1388016000,0,0,0.000000,NaN


# Is there a correlation between the rating of the product and the helpfulness of the review?

When checking the correlation between features we need to take care to assure that pearson correlation makes sense to use. Trying to correlate a variable with a non-ordinal categorical variable doesn't really make sense and co-occurrence should be preferred. However in our case `helpful` (either number of "yes" votes or proportion of "yes" votes) is a numerical variable and `overvall` is ordinal given it has some intrisct order to it (a ratting of 5 is higher than a ratting of 1)